In [1]:
import os
import torch as T
import torchvision as TV
import torchaudio as TA
import cv2
import numpy as np
import random
from tqdm import tqdm
import torch.nn as nn
import torch.nn.functional as F
from torchvision import transforms
from torch import optim
from torch.utils.data import DataLoader, Dataset
import segmentation_models_pytorch as smp
from glob import glob
import albumentations as A
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score, confusion_matrix
from pathlib import Path
import segmentation_models_pytorch as smp

In [2]:
T.manual_seed(32)
np.random.seed(32)
random.seed(32)

In [3]:
# ---------------------- DEVICE -----------------------
device = T.device("cuda" if T.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [4]:
# ---------------------- Paths -----------------------
train_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\train_new"
train_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\trainlabel_new"
validation_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\validation_new"
validation_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\validationlabel_new"
test_images = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\test_new"
test_masks = r"D:\AAU Internship\Code\CWF-788\IMAGE512x384\testlabel_new"

In [7]:
# ---------------------- Simple Transform -----------------------
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# ---------------------- Simplified Dataset Class -----------------------
from glob import glob
import torch
from torch.utils.data import Dataset, DataLoader
import os
import cv2
import numpy as np
from tqdm import tqdm

class SimpleSegmentationDataset(Dataset):
    def __init__(self, image_dir, mask_dir, transform=None, dataset_type="Unknown"):
        self.image_dir = image_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.dataset_type = dataset_type
        self.image_files = sorted(glob(os.path.join(image_dir, "*.jpg")))
        self.mask_files = sorted(glob(os.path.join(mask_dir, "*.png")))
        self._verify_file_pairs()
        
    def _verify_file_pairs(self):
        if len(self.image_files) != len(self.mask_files):
            raise ValueError(f"Mismatched counts in {self.dataset_type} dataset: {len(self.image_files)} images vs {len(self.mask_files)} masks")
            
        for img_path, mask_path in tqdm(zip(self.image_files, self.mask_files), total=len(self.image_files), desc=f"Verifying {self.dataset_type} File Pairs 🔍"):
            img_name = os.path.splitext(os.path.basename(img_path))[0]
            mask_name = os.path.splitext(os.path.basename(mask_path))[0]
            if img_name != mask_name:
                raise ValueError(f"Filename mismatch in {self.dataset_type} dataset: {img_name} vs {mask_name}")
    
    def __len__(self):
        return len(self.image_files)
    
    def __getitem__(self, idx):
        img = cv2.cvtColor(cv2.imread(self.image_files[idx]), cv2.COLOR_BGR2RGB)
        mask = cv2.imread(self.mask_files[idx], cv2.IMREAD_GRAYSCALE)
        mask = (mask > 127).astype(np.uint8)
        if self.transform:
            img = self.transform(img)
        else:
            img = transforms.ToTensor()(img)
        
        mask = T.from_numpy(mask).long()
        
        return img, mask, self.image_files[idx]

def custom_collate_fn(batch):
    images = torch.stack([item[0] for item in batch])
    masks = torch.stack([item[1] for item in batch])
    filenames = [item[2] for item in batch]
    return images, masks, filenames

# ---------------------- DataLoaders -----------------------
train_dataset = SimpleSegmentationDataset(
    image_dir=train_images,
    mask_dir=train_masks,
    transform=transform,
    dataset_type="Training"
)
val_dataset = SimpleSegmentationDataset(
    image_dir=validation_images,
    mask_dir=validation_masks,
    transform=transform,
    dataset_type="Validation"
)
test_dataset = SimpleSegmentationDataset(
    image_dir=test_images,
    mask_dir=test_masks,
    transform=transform,
    dataset_type="Testing"
)

train_dataloader = DataLoader(
    train_dataset,
    batch_size=4,
    shuffle=True,
    num_workers=0,
    pin_memory=True,
    collate_fn=custom_collate_fn
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=custom_collate_fn
)
test_dataloader = DataLoader(
    test_dataset,
    batch_size=4,
    shuffle=False,
    num_workers=0,
    pin_memory=True,
    collate_fn=custom_collate_fn
)

print(f"Training samples: {len(train_dataset)}")
print(f"Validation samples: {len(val_dataset)}")
print(f"Testing samples: {len(test_dataset)}")

Verifying Testing File Pairs 🔍: 100%|█████████████████████████████████████████| 1200/1200 [00:00<00:00, 104548.31it/s]

Training samples: 1600
Validation samples: 352
Testing samples: 1200


In [6]:
# ---------------------- Model -----------------------
CUSTOM_SAVE_ROOT = Path(r"D:\AAU Internship\Code\UNet-Models")
os.makedirs(CUSTOM_SAVE_ROOT, exist_ok=True)

model = smp.Unet(
    encoder="efficientnet-b5",
    encoder_weights="imagenet",
    encoder_depth=4,
    decoder_use_batchnorm='inplace',
    decoder_attention_type='scse',
    decoder_channels=[256, 128, 64, 32],
    in_channels=3,
    classes=2,
    activation=None,
    center=True,
).to(device)

# ---------------------- Loss Function -----------------------
class FocalTverskyLoss(nn.Module):
    def __init__(self, alpha=0.7, beta=0.3, gamma=0.75, smooth=1e-6):
        super().__init__()
        self.alpha = alpha
        self.beta = beta
        self.gamma = gamma
        self.smooth = smooth

    def update_hyperparams_by_epoch(self, epoch):
        steps = epoch // 5
        self.alpha = max(0.4, 0.7 - 0.03*steps)
        self.beta = 1 - self.alpha
        self.gamma = min(1.5, 0.5 + 0.1*steps)

    def forward(self, preds, targets):
        targets_one_hot = F.one_hot(targets, num_classes=preds.shape[1]).permute(0, 3, 1, 2).float()
        probs = F.softmax(preds, dim=1)
        dims = (0, 2, 3)
    
        TP = T.sum(probs * targets_one_hot, dims)
        FP = T.sum(probs * (1 - targets_one_hot), dims)
        FN = T.sum((1 - probs) * targets_one_hot, dims)
    
        Tversky = (TP + self.smooth) / (TP + self.alpha * FP + self.beta * FN + self.smooth)
        return T.mean((1 - Tversky) ** self.gamma)

loss_fn = FocalTverskyLoss().to(device)

# ---------------------- Metrics -----------------------
def compute_metrics(preds, targets):
    with T.no_grad():
        pred_labels = T.argmax(preds, dim=1).cpu().numpy().flatten()
        targets = targets.cpu().numpy().flatten()
        ious = []
        for cls in [0, 1]:
            intersection = ((pred_labels == cls) & (targets == cls)).sum()
            union = ((pred_labels == cls) | (targets == cls)).sum()
            ious.append(intersection / (union + 1e-6))
        class_acc = []
        for cls in [0, 1]:
            mask = (targets == cls)
            if mask.sum() > 0:
                class_acc.append((pred_labels[mask] == cls).mean())
        mPA = np.mean(class_acc) * 100
        cm = confusion_matrix(targets, pred_labels)
        TN, FP, FN, TP = cm.ravel()
        return {
            "Accuracy": 100 * accuracy_score(targets, pred_labels),
            "mPA": mPA,
            "Crop IoU": 100 * ious[1],
            "mIoU": 100 * np.mean(ious),
            "Precision": 100 * precision_score(targets, pred_labels, zero_division=0),
            "Recall": 100 * recall_score(targets, pred_labels, zero_division=0),
            "F1-Score": 100 * f1_score(targets, pred_labels, zero_division=0),
            "FNR": 100 * (FN / (FN + TP + 1e-6))
        }

# ---------------------- Training Setup -----------------------
optimizer = optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-5)
MODEL_PATH = CUSTOM_SAVE_ROOT / "best_mPA_model.pth"
best_mPA = -1  # Initialize to negative value for maximization
PRIMARY_METRIC = "mPA"  # Primary metric for model selection

# ---------------------- Training & Validation -----------------------
def TrainUNet(model, dataloader, loss_fn, optimizer, epoch):
    model.train()
    running_loss = 0
    all_preds, all_targets = [], []
    loss_fn.update_hyperparams_by_epoch(epoch)
    loop = tqdm(dataloader, desc=f"Epoch {epoch} [Train]")

    for batch in loop:
        inputs, targets = batch  # Unpack tuple: images, masks
        inputs = inputs.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = loss_fn(outputs, targets)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        all_preds.append(outputs.detach().cpu())
        all_targets.append(targets.detach().cpu())
        loop.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(dataloader)
    metrics = compute_metrics(T.cat(all_preds), T.cat(all_targets))

    T.cuda.empty_cache()
    
    return avg_loss, metrics

def ValidateUNet(model, dataloader, loss_fn):
    model.eval()
    running_loss = 0
    all_preds, all_targets = [], []
    loop = tqdm(dataloader, desc="Validating")

    with T.no_grad():
        for batch in loop:
            inputs, targets = batch  # Unpack tuple: images, masks
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = loss_fn(outputs, targets)
            running_loss += loss.item()
            all_preds.append(outputs.detach().cpu())
            all_targets.append(targets.detach().cpu())
            loop.set_postfix(loss=loss.item())

    avg_loss = running_loss / len(dataloader)
    metrics = compute_metrics(T.cat(all_preds), T.cat(all_targets))
    
    T.cuda.empty_cache()

    return avg_loss, metrics

# ---------------------- Main Training -----------------------
num_epochs = 50
for epoch in range(1, num_epochs + 1):
    train_loss, train_metrics = TrainUNet(model, train_dataloader, loss_fn, optimizer, epoch)
    val_loss, val_metrics = ValidateUNet(model, val_dataloader, loss_fn)

    # Save model if mPA improves
    if val_metrics[PRIMARY_METRIC] > best_mPA:
        best_mPA = val_metrics[PRIMARY_METRIC]
        T.save(model.state_dict(), str(MODEL_PATH))
        print(f"✅ New best {PRIMARY_METRIC}: {best_mPA:.2f}% | Saved to: {MODEL_PATH}")

    # Print epoch summary
    print(f"\n📊 Epoch {epoch} Summary:")
    print(f"Train Loss: {train_loss:.4f} | Val Loss: {val_loss:.4f}")
    for k, v in val_metrics.items():
        print(f"{k}: {v:.2f}%")

    T.cuda.empty_cache()

# ---------------------- Final Report -----------------------
print(f"\n🎯 === Best Model Summary ===")
print(f"Best {PRIMARY_METRIC}: {best_mPA:.2f}% → {MODEL_PATH}")

# ---------------------- Testing -----------------------
print("\n🧪 === Testing Best Model ===")
model.load_state_dict(T.load(str(MODEL_PATH)))
test_loss, test_metrics = ValidateUNet(model, test_dataloader, loss_fn)
print(f"\n📌 Best {PRIMARY_METRIC} Model Test Results:")
for k, v in test_metrics.items():
    print(f"{k}: {v:.2f}%")

Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:23<00:00,  3.82it/s, loss=0.163]


✅ New best mPA: 97.53% | Saved to: D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

📊 Epoch 1 Summary:
Train Loss: 0.2873 | Val Loss: 0.1488
Accuracy: 99.17%
mPA: 97.53%
Crop IoU: 92.28%
mIoU: 95.67%
Precision: 96.51%
Recall: 95.46%
F1-Score: 95.98%
FNR: 4.54%


Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:21<00:00,  4.17it/s, loss=0.133]


✅ New best mPA: 98.57% | Saved to: D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

📊 Epoch 2 Summary:
Train Loss: 0.1368 | Val Loss: 0.1226
Accuracy: 99.37%
mPA: 98.57%
Crop IoU: 94.15%
mIoU: 96.72%
Precision: 96.42%
Recall: 97.56%
F1-Score: 96.99%
FNR: 2.44%


Validating: 100%|██████████████████████████████████████████████████████████| 88/88 [00:21<00:00,  4.08it/s, loss=0.119]


...


📊 Epoch 49 Summary:
Train Loss: 0.0012 | Val Loss: 0.0031
Accuracy: 99.47%
mPA: 98.80%
Crop IoU: 95.11%
mIoU: 97.26%
Precision: 97.05%
Recall: 97.94%
F1-Score: 97.49%
FNR: 2.06%


Validating: 100%|████████████████████████████████████████████████████████| 88/88 [00:21<00:00,  4.01it/s, loss=0.00228]



📊 Epoch 50 Summary:
Train Loss: 0.0007 | Val Loss: 0.0021
Accuracy: 99.47%
mPA: 98.77%
Crop IoU: 95.07%
mIoU: 97.24%
Precision: 97.07%
Recall: 97.88%
F1-Score: 97.47%
FNR: 2.12%

🎯 === Best Model Summary ===
Best mPA: 98.95% → D:\AAU Internship\Code\UNet-Models\best_mPA_model.pth

🧪 === Testing Best Model ===


Validating: 100%|██████████████████████████████████████████████████████| 300/300 [01:24<00:00,  3.55it/s, loss=0.00406]



📌 Best mPA Model Test Results:
Accuracy: 99.50%
mPA: 98.91%
Crop IoU: 94.74%
mIoU: 97.09%
Precision: 96.43%
Recall: 98.19%
F1-Score: 97.30%
FNR: 1.81%


In [ ]:
# ---------------------- Mask Generation -----------------------
from pathlib import Path
from tqdm import tqdm

MASK_FOLDER_NAME = "Crop_Masks"
MASK_OUTPUT_DIR = Path.cwd() / MASK_FOLDER_NAME
for split in ['train', 'val', 'test']:
    (MASK_OUTPUT_DIR / split).mkdir(parents=True, exist_ok=True)

def save_segmentation_masks(model, train_dataloader, val_dataloader, test_dataloader, model_path, output_dir, device):
    model.load_state_dict(T.load(str(model_path)))
    model.eval()
    dataloader_splits = [
        (train_dataloader, 'train'),
        (val_dataloader, 'val'),
        (test_dataloader, 'test')
    ]
    mask_counter = 0
    
    with T.no_grad():
        for dataloader, split in tqdm(dataloader_splits, desc="Processing datasets"):
            split_dir = output_dir / split
            for batch in tqdm(dataloader, desc=f"Processing {split} batch", leave=False):
                inputs, _, filenames = batch
                inputs = inputs.to(device)
                batch_size = inputs.size(0)
                outputs = model(inputs)
                pred_labels = T.argmax(outputs, dim=1).cpu().numpy()
                for i in range(batch_size):
                    mask = pred_labels[i]
                    mask = (mask * 255).astype(np.uint8)
                    image_name = Path(filenames[i]).name
                    mask_path = str(split_dir / image_name)
                    cv2.imwrite(mask_path, mask)
                    mask_counter += 1
    
    print(f"\n🎉 Saved {mask_counter} segmentation masks to {output_dir}")

print("\n🖼️ Generating and saving segmentation masks...")
save_segmentation_masks(
    model=model,
    train_dataloader=train_dataloader,
    val_dataloader=val_dataloader,
    test_dataloader=test_dataloader,
    model_path=CUSTOM_SAVE_ROOT / "best_mPA_model.pth",
    output_dir=MASK_OUTPUT_DIR,
    device=device
)

T.cuda.empty_cache()


🖼️ Generating and saving segmentation masks...


Processing train batch:  46%|██████████████████████████                              | 186/400 [00:51<00:54,  3.94it/s]